In [1]:
import sys
sys.path.extend(["../../../../"])

In [2]:
import numpy as np
from onsager.crystal import Crystal
from onsager.OnsagerCalc import dumbbellMediated
from onsager.crystal import DB_disp, DB_disp4, pureDBContainer, mixedDBContainer
from onsager.DB_structs import dumbbell, SdPair, jump, connector

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3.1-py3.6.egg/onsager/cluster.py:804: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
# make a BCC lattice
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = crystal.Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110]
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = pureDBContainer(Fe, 0, family)
mdbcontainer_fe = mixedDBContainer(Fe, 0, family)
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01)

In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to smallest net atomic displacement
def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = DB_disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
#     if np.allclose(jnet0_indexed[jt][0][1], z):
#         continue
    indices.append(jt)

ind_sort = sorted(indices, key=sortkey)[:3]
len(ind_sort)

3

In [5]:
# we'll consider all the pure dumbbell jumps mentioned in the database,
# They are: 60-degree roto-translation, rigid translation and 60-degree on-site rotation
for ind in ind_sort:
    jmp = jnet0[ind][0]
    print(jmp)
    print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	dumbbell : (i, or) index = 2, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 4, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = 1

[-0.14155  0.14155  0.14155]

Jump object:
Initial state:
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 1, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = -1

[0. 0. 0.]

Jump object:
Initial state:
	dumbbell : (i, or) index = 2, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 2, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = -1

[-0.14155  0.14155  0.14155]



In [6]:
pdbcontainer_fe.iorlist

[(0, array([-1.78190909e-01,  2.83727636e-34, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([-1.78190909e-01,  1.78190909e-01, -5.09739416e-18])),
 (0, array([-2.83727636e-34, -1.78190909e-01,  1.78190909e-01])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01]))]

In [7]:
# take the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the database of Messina et. al.
# They are: 60 deg rot-trans, rigid translation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want only the rotational jumps as listed.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
indices_rot = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        jmp = jlist[0]

        or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
        or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
        
        if np.allclose(np.dot(or1,or2)/(np.linalg.norm(or1)*np.linalg.norm(or2)), np.cos(np.pi/3.)):
#             print("got 60 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        elif np.allclose(np.dot(or1,or2), 0.):
#             print("got 90 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        else:
            continue
    indices2.append(jt)
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:2]
# indices2all.append(indices_rot[0])
print(indices2all, indices_rot)

[15, 16] [18, 19]


In [9]:
# check if we have the correct type of jumps
for ind in indices2all:
    jmp = jnet2[ind][0]
    print(jmp)
#     print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 2, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 10, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1



In [10]:
print(mdbcontainer_fe.iorlist[11])
print(mdbcontainer_fe.iorlist[1])
print(mdbcontainer_fe.iorlist[10])

(0, array([ 1.78190909e-01, -1.78190909e-01,  5.09739416e-18]))
(0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17]))
(0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01]))


In [11]:
# take only the lowest displacement jump
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])
len(jset0new[0]), len(jset2new[0])

(3, 2)

In [12]:
# re-initialize the calculator.
# Now, we construct the Onsager calculator with these non-local jump sets.
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.26,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
generating thermodynamic shell
built shell 1: time - 0.050925493240356445
built shell 2: time - 1.5414462089538574
built shell 3: time - 7.599425554275513
grouped states by symmetry: 6.39967942237854
built mixed dumbbell stars: 0.0004286766052246094
built jtags2: 0.0005645751953125
built mixed indexed star: 0.007628202438354492
building star2symlist : 0.00013065338134765625
building bare, mixed index dicts : 0.000232696533203125
thermodynamic shell generated: 21.896941900253296
Total number of states in Thermodynamic Shell - 546, 12
generating kinetic shell
built shell 1: time - 0.055747270584106445
built shell 2: time - 1.9055471420288086
built shell 3: time - 8.121885299682617
built shell 4: time - 19.515453577041626
grouped states by symmetry: 25.338582038879395
built mixed dumbbell stars: 0.0004246234893798828
built jtags2: 0.0009341239929199219
built mixed indexed star: 0.005930662155151367
building star2symlist : 0.0002105236053466797
building

### Next, we must extract the necessary omega43 data

In [13]:
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# We assume that association and dissociation occurs via Johnson mechanism.

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[18]


In [14]:
print(jnet43[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 5, lattice vector = [-1 -1 -1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 10, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = -1


In [15]:
pdbcontainer_fe.iorlist

[(0, array([-1.78190909e-01,  2.83727636e-34, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([-1.78190909e-01,  1.78190909e-01, -5.09739416e-18])),
 (0, array([-2.83727636e-34, -1.78190909e-01,  1.78190909e-01])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01]))]

In [16]:
mdbcontainer_fe.iorlist

[(0, array([-1.78190909e-01,  2.83727636e-34, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([-1.78190909e-01,  1.78190909e-01, -5.09739416e-18])),
 (0, array([-2.83727636e-34, -1.78190909e-01,  1.78190909e-01])),
 (0, array([ 2.83727636e-34,  1.78190909e-01, -1.78190909e-01])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([-1.78190909e-01, -1.78190909e-01,  1.01947883e-17])),
 (0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01])),
 (0, array([ 1.78190909e-01,  2.00863708e-17, -1.78190909e-01])),
 (0, array([ 1.78190909e-01, -1.97831650e-17,  1.78190909e-01])),
 (0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01])),
 (0, array([ 1.78190909e-01, -1.78190909e-01,  5.09739416e-18]))]

In [17]:
print(onsagercalculator.crys)

#Lattice:
  a1 = [0.14155 0.14155 0.14155]
  a2 = [-0.14155  0.14155 -0.14155]
  a3 = [-0.14155 -0.14155  0.14155]
#Basis:
  (Fe) 0.0 = [0. 0. 0.]


In [18]:
onsagercalculator.regenerate43(ind_sort43)

In [19]:
import pickle
with open('Fe_Cu_Ni_Si.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)